In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import tensorflow as tf
import joblib
import time

In [3]:
# Price for zero-coupon bond with stochastic interest rate under Vasicek's model
def ZC_Vasicek(F, r, kappa, theta, sigma, t, T):
    
    delta_T = T - t
    
    B = (1 - np.exp(-kappa * delta_T)) / kappa
    
    A = np.exp((theta - (sigma**2) / (2 * kappa**2)) * (B - delta_T) - (sigma**2 / (4 * kappa)) * B**2)
    
    bond_price = F * A * np.exp(-B * r)
    
    return bond_price

In [4]:
def rpoi(a, y, lam, t, T):
    return np.exp(lam * (T-t) * (np.exp(a) - 1) - a * y)

def rgamma(a, y, k, th, n):
    return (1-th*a)**(-k*n) / np.exp(a * y)

In [5]:
# MC with gamma distribution for the size of losses
def naive_MC_gamma(nr, lam, D, k, th, T):
    h = []
    poissons = np.random.poisson(lam=lam*T, size=nr)
    for i in range(nr):
        x = np.sum(np.random.gamma(shape=k, scale=th, size=poissons[i]))
        h.append(int(x>D))
    return np.cumsum(h)/np.arange(1,nr+1)

In [6]:
# Importance sampling for default probability
def MC_IS_gamma_poi(nr, lam, D, k, th, t, T):
    # Initial variables
    a_poi = np.log(D / (lam * (T-t) * k * th))/2
    poisson_means = lam * (T-t) * np.exp(a_poi)
    a_gamma = 1/th - (k * poisson_means)/D
    new_th = 1/(1/th - a_gamma)

    # Preallocate memory for cumulative sums
    h = np.zeros(nr)
    r_poi = np.zeros(nr)
    r_gamma = np.zeros(nr)
    
    # Loop over the number of simulations
    for i in range(nr):
        # Generate Poisson-distributed count
        loss_n = np.random.poisson(lam=poisson_means)

        # Generate the exponential random variables for this count
        # Directly sum them without creating large intermediate arrays
        x = np.sum(np.random.gamma(shape=k, scale=new_th, size=loss_n))

        # Compute h and r for this simulation
        h[i] = int(x > D)
        r_poi[i] = rpoi(a_poi, loss_n, lam, t, T)
        r_gamma[i] = rgamma(a_gamma, x, k, th, loss_n)

    # Calculate cumulative sum and return the average at each step
    cumulative_sum = np.cumsum(h * r_poi * r_gamma)
    cumulative_avg = cumulative_sum / np.arange(1, nr + 1)

    return cumulative_avg

In [7]:
# Zero-coupon CAT bond pricing under IS
def IS_CAT_ZC_Vasicek(F, r, kappa, theta, sigma, D, lam, k, th, nr, t, T):
    return ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_gamma_poi(nr, lam, D, k, th, t, T)[-1])

In [8]:
# CAT bond with coupons pricing
def CAT_C_Vasicek(D, lam, N, T):
    c_sum = 0
    dt = T/N

    for coupon_count in range (N):
        if lam * (coupon_count+1) * dt * k * th < D:
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-MC_IS_gamma_poi(nr, lam, D, k, th, t, T=(coupon_count+1)*dt)[-1])
        else:  # Case: N == 0 and lam * T * k * th > D
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-naive_MC_gamma(nr, lam, D, k, th, T=(coupon_count+1)*dt)[-1])
    
    if lam * T * k * th < D:
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_gamma_poi(nr, lam, D, k, th, t, T)[-1])
    else:  # Case: N == 0 and lam * T * k * th > D
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-naive_MC_gamma(nr, lam, D, k, th, T)[-1])
    
    return c_sum

In [16]:
# Parameter ranges
F = 1  # Fixed face value
t = 0     # Fixed initial time
k = 1     # Fixed Gamma shape
th = 163500000    # Fixed Gamma scale
c=0.05
r=0.03
kappa=0.2
theta=0.03
sigma=0.02
nr = 10000

D=9*1000000000
lam=35
N=2
T=1

num_runs = 1000  # Number of iterations
results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for prediction: {total_time_MC:.6f} seconds")

Prediction: 1.053338143812643
Time taken for prediction: 132.809459 seconds


In [17]:
# Define custom objects dictionary
custom_objects = {"mse": tf.keras.losses.MeanSquaredError()}

# Load the saved model
model_gamma = tf.keras.models.load_model("best_NN_gamma.h5", custom_objects=custom_objects)

# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 2, 1]])  # Modify based on model's input shape

# Load the fitted scaler
scaler = joblib.load("scaler_gamma.pkl")

# Apply the same scaling
new_input_scaled = scaler.transform(new_input)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_gamma.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Prediction: 1.0533469
Time taken for prediction: 0.109503 seconds


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 2, 1]])  # Modify based on model's input shape

# Apply the same scaling
new_input_scaled = scaler.transform(new_input)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_gamma.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step
Prediction: 1.0533469
Time taken for prediction: 0.040404 seconds


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [22]:
N=4
T=1

results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for prediction: {total_time_MC:.6f} seconds")

Prediction: 1.1518498323393265
Time taken for prediction: 220.298800 seconds


In [23]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 4, 1]])  # Modify based on model's input shape

# Apply the same scaling
new_input_scaled = scaler.transform(new_input)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_gamma.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step
Prediction: 1.150935
Time taken for prediction: 0.042986 seconds


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [24]:
N=8
T=2

results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for prediction: {total_time_MC:.6f} seconds")

Prediction: 0.37830807636013103
Time taken for prediction: 342.129541 seconds


In [25]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 8, 2]])  # Modify based on model's input shape

# Apply the same scaling
new_input_scaled = scaler.transform(new_input)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_gamma.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 450us/step
Prediction: 0.38236773
Time taken for prediction: 0.032882 seconds


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [26]:
N=12
T=2

results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for prediction: {total_time_MC:.6f} seconds")

Prediction: 0.5330722854755693
Time taken for prediction: 438.201514 seconds


In [27]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 12, 2]])  # Modify based on model's input shape

# Apply the same scaling
new_input_scaled = scaler.transform(new_input)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_gamma.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step
Prediction: 0.5309458
Time taken for prediction: 0.031105 seconds


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
